
# Week03 — Prompt Evaluation & Version Management (Langfuse + Notion + GitHub)

이 노트북은 **뉴스 기사 10개**를 입력으로 받아,  
- **V0.0.1 (간단 요약 Prompt)**, **V0.0.2 (구조화된 뉴스 요약 Prompt)** 두 번의 Prompt Engineering을 수행하고,  
- **Langfuse Tracing**을 남기며,  
- **Langfuse Datasets**에 평가용 샘플 10개를 업로드하고,  
- 이후 **Dataset Run/Evaluation**을 할 수 있도록 기본 코드를 제공합니다.

> ⚙️ 이 노트북은 `.env`에 저장된 다음 변수들을 사용합니다.
> - `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_HOST` (예: `https://cloud.langfuse.com` 또는 리전 URL)
> - `OPENAI_API_KEY` (또는 호환 LLM API 키)


In [5]:
%pip install -q langfuse python-dotenv openai requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
# --- 0) 설치 & 기본 설정 ---------------------------------------------------------
# 인터넷 환경에 따라 설치가 제한될 수 있습니다. 로컬 환경에서 실행하세요.
# %pip install -q langfuse python-dotenv openai requests

import os, json, time, uuid, sys
from pathlib import Path
from datetime import datetime
from typing import Dict, Any

# .env 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env")
except Exception as e:
    print("python-dotenv not available; make sure environment variables are set.")

LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("Langfuse host:", LANGFUSE_HOST)
print("Langfuse public key set? ", bool(LANGFUSE_PUBLIC_KEY))
print("Langfuse secret key set? ", bool(LANGFUSE_SECRET_KEY))
print("OpenAI key set? ", bool(OPENAI_API_KEY))


Loaded .env
Langfuse host: https://cloud.langfuse.com
Langfuse public key set?  True
Langfuse secret key set?  True
OpenAI key set?  True


In [7]:

# --- 1) 뉴스 기사 로드 ------------------------------------------------------
news_dir = Path("datasets/news")
news_files = sorted(list(news_dir.glob("news_*.txt")))

# 첫 번째 뉴스 기사를 데모용으로 로드
sample_news_text = ""
if news_files:
    try:
        with open(news_files[0], "r", encoding="utf-8") as f:
            sample_news_text = f.read()
            print("Loaded sample news:", news_files[0].name)
            print("Length:", len(sample_news_text))
            print("\n--- Preview (first 800 chars) ---\n")
            print(sample_news_text[:800])
    except Exception as e:
        print("Error loading sample news:", e)
        sample_news_text = "LG가 런던증권거래소그룹(LSEG)와 손잡고 독자 인공지능(AI) 모델 '엑사원'(EXAONE)을 기반으로 한 '금융 AI 에이전트'의 상용화에 나섰다... (샘플 뉴스)"
else:
    print("No news files found in datasets/news directory")
    sample_news_text = "LG가 런던증권거래소그룹(LSEG)와 손잡고 독자 인공지능(AI) 모델 '엑사원'(EXAONE)을 기반으로 한 '금융 AI 에이전트'의 상용화에 나섰다... (샘플 뉴스)"


Loaded sample news: news_01.txt
Length: 1408

--- Preview (first 800 chars) ---

LG가 런던증권거래소그룹(LSEG)와 손잡고 독자 인공지능(AI) 모델 ‘엑사원’(EXAONE)을 기반으로 한 ‘금융 AI 에이전트’의 상용화에 나섰다. 이번 협업은 양국간 첫 ‘금융 AI’ 협력 사례로, 한국의 AI 기술력이 글로벌 시장으로 확산하는 계기가 될 것으로 전망된다.

LG는 자사 AI연구원과 LSEG가 지난 19일(현지시각) 영국 런던증권거래소에서 금융 AI 에이전트 ‘엑사원 비즈니스 인텔리전스’ 상용화 서비스 시작을 알리는 행사를 진행했다고 21일 밝혔다.

이날 행사에는 토드 하트만 데이터·피드 그룹 총괄, 이보 데커스 유럽·중동·아프리카 영업 그룹 총괄, 사이먼 유든 퀀트·데이터 총괄, 앤드류 파이프 아시아태평양 지역 영업 총괄을 비롯한 LSEG 경영진과 이홍락 공동 연구원장, 임우형 공동 연구원장, 이화영 AI사업개발부문장 등 LG AI연구원 경영진이 참석했다.

토드 하트만 총괄은 “LG와의 파트너십은 새로운 시작을 의미한다”며 “AI는 전 세계 투자자들에게 예측부터 의사결정 지원에 이르기까지 더 나은 기회를 제공하는 핵심적인 역할을 할 것”이라고 말했다.

LSEG는 글로벌 금융 인프라·데이터 분야를 선도하는 영국 대표 금융 기업으로, 런던증권거래소를 운영하고 있다.

LSEG는 금융 시장의 방대한 데이터와 이를 분석한 자료를 전 세계 투자자들에게 제공하는 것을 핵심 사업으로 삼고 있다. 이 핵심 사업에 LG AI연구원의 금융 AI 에이전트 ‘엑사원-BI’를 도입하는 것이다.

‘엑사원-BI’는 인간 개입 없이 AI가 데이터 분석부터 미래 예측, 보고서 작성까지 전 과정을 수행하는 ‘



## 2) Prompts — V0.0.1 (간단 요약) vs V0.0.2 (구조화 회의록)

- **V0.0.1**: 간단 요약
- **V0.0.2**: 자세한 요약


In [12]:

V001_SIMPLE_SUMMARY = '''You are a news summarizer.
Summarize the following news article in **one short paragraph (max 120 words)**.
Focus only on the most important facts: who, what, when, where, and why.
Do not add opinions or speculation.

Write in 'Korean'
'''

V002_STRUCTURED_MINUTES = '''You are an expert news analyst.
Summarize the following news article into a **structured Markdown report** with the following sections:

# Headline
- A concise title (≤15 words) that captures the main story

# Summary
- 1–2 paragraphs describing the main events in neutral tone

# Key Points
- 3–5 bullet points with the most critical facts (who, what, when, where, why, how)

# Implications
- 2–3 sentences on potential impacts (political, economic, or social)

# Quotes
- Up to 2 direct quotes if available in the article

Rules:
- Remain objective, no speculation beyond the article content.
- Do not exceed 300 words in total.
- Preserve named entities (people, organizations, places) accurately.
- Write in 'Korean'
'''


In [13]:

# --- 3) LLM 호출 추상화 -----------------------------------------------------------
def call_openai_chat(system_prompt: str, user_text: str, model: str = "gpt-4o-mini", temperature: float = 0.2) -> str:
    """LLM 호출. OPENAI_API_KEY가 없으면 모의 출력(mock)으로 대체."""
    if os.getenv("OPENAI_API_KEY"):
        try:
            # Try official openai package first
            try:
                from openai import OpenAI
                client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
                resp = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_text},
                    ],
                    temperature=temperature,
                )
                return resp.choices[0].message.content
            except Exception:
                # Fallback to raw HTTP if new package not available
                import requests
                headers = {
                    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
                    "Content-Type": "application/json",
                }
                payload = {
                    "model": model,
                    "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_text},
                    ],
                    "temperature": temperature,
                }
                r = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload, timeout=60)
                r.raise_for_status()
                data = r.json()
                return data["choices"][0]["message"]["content"]
        except Exception as e:
            print("OpenAI call failed, falling back to mock. Error:", e)
    # Mock output for offline demo
    return f"""[MOCK OUTPUT]\nSystem: {system_prompt.splitlines()[0]}\nUserInputPreview: {user_text[:120]}...\n- Bullet 1\n- Bullet 2\n- Bullet 3"""


In [10]:

# --- 4) Langfuse 초기화 & Tracing 유틸 -------------------------------------------
USE_SDK = False
langfuse = None

try:
    from langfuse import get_client, observe
    from langfuse.openai import OpenAI as LFOpenAI  # optional
    langfuse = get_client()
    if langfuse and langfuse.auth_check():
        USE_SDK = True
        print("Langfuse SDK authenticated.")
    else:
        print("Langfuse SDK not authenticated; will use HTTP or no-op.")
except Exception as e:
    print("Langfuse SDK not available; continuing without it.", e)

def start_trace(name: str, metadata: Dict[str, Any] = None):
    if USE_SDK:
        # Start span as root trace context manager
        return langfuse.start_as_current_span(name=name, metadata=metadata or {})
    else:
        # No-op context manager
        from contextlib import contextmanager
        @contextmanager
        def _noop():
            class Dummy:
                def score_trace(self, *args, **kwargs): pass
                def update_trace(self, *args, **kwargs): pass
            yield Dummy()
        return _noop()

def log_trace_io(input_obj: Any, output_obj: Any):
    if USE_SDK:
        try:
            # update current trace input/output to enable dataset evals linkage
            langfuse.update_current_trace(input=input_obj, output=output_obj)
        except Exception as e:
            print("Failed to update current trace:", e)

def flush_langfuse():
    try:
        if USE_SDK:
            langfuse.flush()
    except Exception:
        pass


Langfuse SDK authenticated.


In [14]:

# --- 5) 두 가지 Prompt 실행 + Tracing --------------------------------------------
subset = sample_news_text[:4000]  # 데모를 위해 앞부분만 사용

results = {}

with start_trace(name="news-summary:v0.0.1", metadata={"version": "v0.0.1", "label": "dev", "use_case":"news-summary"}) as span:
    out1 = call_openai_chat(V001_SIMPLE_SUMMARY, subset)
    results["v0.0.1"] = out1
    log_trace_io({"news_article": subset, "prompt_version": "v0.0.1"}, {"summary": out1})

with start_trace(name="news-summary:v0.0.2", metadata={"version": "v0.0.2", "label": "staging", "use_case":"news-summary"}) as span:
    out2 = call_openai_chat(V002_STRUCTURED_MINUTES, subset)
    results["v0.0.2"] = out2
    log_trace_io({"news_article": subset, "prompt_version": "v0.0.2"}, {"summary": out2})

flush_langfuse()

print("\n=== V0.0.1 (simple) ===\n", results["v0.0.1"][:800], "...")
print("\n=== V0.0.2 (structured) ===\n", results["v0.0.2"][:800], "...")



=== V0.0.1 (simple) ===
 LG는 런던증권거래소그룹(LSEG)과 협력하여 독자 인공지능 모델 '엑사원'(EXAONE)을 기반으로 한 '금융 AI 에이전트'의 상용화를 시작했다. 이 협업은 한국의 AI 기술을 글로벌 시장에 확산시키는 첫 사례로, 19일 런던증권거래소에서 상용화 서비스 시작을 알리는 행사가 열렸다. '엑사원-BI'는 AI가 데이터 분석, 예측, 보고서 작성 등을 수행하며, LSEG는 이를 통해 전 세계 투자자들에게 데이터 상품을 판매할 예정이다. LG AI연구원은 이 AI 에이전트가 새로운 AI 시대를 여는 시작점이 될 것으로 기대하고 있다. ...

=== V0.0.2 (structured) ===
 # Headline
LG, LSEG와 협력해 금융 AI 에이전트 '엑사원' 상용화

# Summary
LG가 런던증권거래소그룹(LSEG)과 협력하여 독자 인공지능(AI) 모델 '엑사원'(EXAONE)을 기반으로 한 '금융 AI 에이전트'의 상용화에 나섰다. 이 협업은 한국의 AI 기술이 글로벌 시장으로 확산하는 계기가 될 것으로 기대되며, 지난 19일 영국 런던증권거래소에서 상용화 서비스 시작을 알리는 행사가 개최됐다.

행사에는 LSEG와 LG AI연구원 경영진이 참석했으며, LSEG의 토드 하트만 총괄은 LG와의 파트너십이 AI의 핵심 역할을 강조했다. '엑사원-BI'는 AI가 데이터 분석부터 예측, 보고서 작성까지 전 과정을 수행하는 금융 AI 에이전트로, LSEG는 이를 통해 전 세계 투자자들에게 데이터 상품을 판매할 예정이다.

# Key Points
- LG와 LSEG가 협력하여 '엑사원-BI' 금융 AI 에이전트를 상용화.
- 상용화 발표는 19일 런던증권거래소에서 진행됨.
- '엑사원-BI'는 AI가 데이터 분석과 예측을 수행하는 시스템.
- LSEG는 이 AI 에이전트를 통해 데이터 상품 'AEFS'를 판매할 계획.
- LG AI연구원은 이 협력이 AI를 활용한 수익 창출의 시작점이 될 것이라고 기대.

# Im


## 6) Langfuse Dataset 생성 & 아이템 업로드 (뉴스 10개)

- 뉴스 기사 10개를 input으로, 기대 구조를 reference로 저장합니다.  
- 이후 Langfuse에서 **Dataset Run**을 실행하여, 각 Prompt 버전의 결과를 비교/평가할 수 있습니다.


In [15]:

# Dataset 이름
DATASET_NAME = "week03_news_summary_demo"

# 뉴스 파일들 로드
news_dir = Path("datasets/news")
news_files = sorted(list(news_dir.glob("news_*.txt")))
print(f"Found {len(news_files)} news files")

# 뉴스 파일들을 읽어서 데이터셋 아이템으로 변환
news_items = []
for news_file in news_files:
    try:
        with open(news_file, "r", encoding="utf-8") as f:
            content = f.read().strip()
            if content:  # 빈 파일이 아닌 경우만 추가
                news_items.append({
                    "input": {"news_article": content},
                    "expected_output": {
                        "sections": ["Headline", "Summary", "Key Points", "Implications", "Quotes"],
                        "word_count_limit": 300
                    }
                })
                print(f"Loaded {news_file.name}: {len(content)} characters")
    except Exception as e:
        print(f"Error loading {news_file.name}: {e}")

print(f"\nTotal news items for dataset: {len(news_items)}")

created = False
if USE_SDK:
    try:
        langfuse.create_dataset(name=DATASET_NAME)
        for item in news_items:
            langfuse.create_dataset_item(
                dataset_name=DATASET_NAME,
                input=item["input"],
                expected_output=item["expected_output"]
            )
        created = True
        print(f"Created dataset '{DATASET_NAME}' with {len(news_items)} items via SDK.")
    except Exception as e:
        print("SDK dataset creation failed:", e)

# 로컬 JSONL도 함께 저장(백업/수동 업로드용)
ds_dir = Path("datasets"); ds_dir.mkdir(exist_ok=True)
jsonl_path = ds_dir / f"{DATASET_NAME}.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in news_items:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
print("Local backup dataset written to", jsonl_path.resolve())


Found 10 news files
Loaded news_01.txt: 1406 characters
Loaded news_02.txt: 1231 characters
Loaded news_03.txt: 1057 characters
Loaded news_04.txt: 764 characters
Loaded news_05.txt: 1342 characters
Loaded news_06.txt: 1156 characters
Loaded news_07.txt: 1476 characters
Loaded news_08.txt: 1885 characters
Loaded news_09.txt: 989 characters
Loaded news_10.txt: 2064 characters

Total news items for dataset: 10
Created dataset 'week03_news_summary_demo' with 10 items via SDK.
Local backup dataset written to /Users/hajuheon/Task/ai/ajou/25-2/llmops/week03/datasets/week03_news_summary_demo.jsonl



## 7) (선택) Native Dataset Run 실행 예시

아래 코드는 Langfuse **Datasets Cookbook**의 패턴을 따라, 각 Dataset Item을 순회하며 애플리케이션을 실행하고  
`root_span.score_trace(...)` 등으로 평가 스코어를 기록합니다.  
실행 전, SDK 인증이 되어 있어야 합니다.



In [18]:
def analyze_evaluation_results():
    """
    Langfuse에서 평가 결과를 분석하고 요약 통계를 출력합니다.
    """
    if not USE_SDK:
        print("Langfuse SDK unavailable; cannot analyze results.")
        return
    
    try:
        # 데이터셋 정보 가져오기
        dataset = langfuse.get_dataset(DATASET_NAME)
        print(f"\n📊 Dataset Analysis: {DATASET_NAME}")
        print(f"Total items: {len(dataset.items)}")
        
        # 각 실행에 대한 통계 계산
        runs = ["news_summary_v0.0.1", "news_summary_v0.0.2"]
        
        for run_name in runs:
            print(f"\n🔍 Run: {run_name}")
            
            # 해당 실행의 모든 스코어 수집
            scores = {
                'section_coverage': [],
                'summary_quality': [],
                'overall_score': []
            }
            
            for item in dataset.items:
                # 각 아이템의 실행 결과에서 스코어 추출
                # 실제 구현에서는 Langfuse API를 통해 스코어 데이터를 가져와야 함
                pass
            
            print(f"  📈 Scores will be displayed after evaluation completion")
            
    except Exception as e:
        print(f"Error analyzing results: {e}")

# 평가 결과 분석 (선택적 실행)
# analyze_evaluation_results()


## 📊 LLM-as-a-Judge 평가 시스템

위의 코드는 뉴스 요약 품질을 평가하는 **LLM-as-a-Judge** 시스템을 구현합니다.

### 평가 지표:

1. **`section_coverage`** (구조적 완성도, 30% 가중치)
   - 요약이 예상 섹션들(Headline, Summary, Key Points, Implications, Quotes)을 얼마나 잘 포함하는지 평가

2. **`summary_quality`** (LLM 품질 평가, 70% 가중치)
   - 핵심 정보 보존도 (30%)
   - 요약 정확성 (25%) 
   - 구조적 완성도 (20%)
   - 정보 압축도 (15%)
   - 객관성 (10%)

3. **`overall_score`** (종합 점수)
   - 구조적 완성도 × 0.3 + 품질 평가 × 0.7

### 사용법:
1. 위의 셀을 실행하여 두 프롬프트 버전을 평가합니다
2. Langfuse 대시보드에서 상세한 결과를 확인할 수 있습니다
3. 각 뉴스 기사별로 세 가지 점수가 기록됩니다


In [19]:

def presence_of_sections_eval(output_text: str, expected_sections) -> float:
    """출력에 예상 섹션 헤더가 얼마나 포함되는지 0.0~1.0 반환."""
    if not output_text:
        return 0.0
    hits = 0
    for sec in expected_sections:
        if f"# {sec}" in output_text or sec in output_text:
            hits += 1
    return hits / max(1, len(expected_sections))

def llm_judge_summary_quality(original_text: str, summary_text: str) -> float:
    """
    LLM-as-a-judge 방식으로 뉴스 요약의 품질을 평가합니다.
    0.0~1.0 스케일로 점수를 반환합니다.
    """
    if not original_text or not summary_text:
        return 0.0
    
    evaluation_prompt = f"""당신은 뉴스 요약 품질 평가 전문가입니다. 
다음 원문과 요약을 비교하여 요약의 품질을 평가해주세요.

평가 기준:
1. 핵심 정보 보존도 (30%): 원문의 중요한 사실, 인물, 시간, 장소, 숫자 등이 얼마나 잘 보존되었는가?
2. 요약 정확성 (25%): 요약된 내용이 원문과 일치하는가? 잘못된 정보나 왜곡이 없는가?
3. 구조적 완성도 (20%): 요약이 논리적이고 읽기 쉽게 구성되었는가?
4. 정보 압축도 (15%): 원문의 핵심만을 효율적으로 압축했는가?
5. 객관성 (10%): 요약이 중립적이고 객관적인가? 개인 의견이나 추측이 없는가?

원문:
{original_text[:2000]}

요약:
{summary_text}

평가 점수를 0.0에서 1.0 사이의 실수로만 응답해주세요. 
추가 설명 없이 숫자만 출력하세요.
예: 0.85"""

    try:
        score_text = call_openai_chat(
            system_prompt="당신은 뉴스 요약 품질 평가 전문가입니다. 주어진 기준에 따라 객관적이고 정확한 평가를 해주세요.",
            user_text=evaluation_prompt,
            model="gpt-4o-mini",
            temperature=0.1
        )
        
        # 점수 추출 (숫자만 추출)
        import re
        score_match = re.search(r'(\d+\.?\d*)', score_text.strip())
        if score_match:
            score = float(score_match.group(1))
            # 0-1 범위로 정규화
            if score > 1.0:
                score = score / 10.0 if score <= 10.0 else 1.0
            return max(0.0, min(1.0, score))
        else:
            return 0.5  # 파싱 실패 시 중간값 반환
            
    except Exception as e:
        print(f"LLM judge evaluation failed: {e}")
        return 0.5  # 오류 시 중간값 반환

def run_dataset_experiment(run_name: str, system_prompt: str):
    if not USE_SDK:
        print("Langfuse SDK unavailable; skipping remote run.")
        return
    dataset = langfuse.get_dataset(DATASET_NAME)
    total_items = len(dataset.items)
    
    print(f"Starting dataset run '{run_name}' with {total_items} items...")
    
    for i, item in enumerate(dataset.items, 1):
        print(f"Processing item {i}/{total_items}...")
        
        with item.run(run_name=run_name) as root_span:
            # 뉴스 요약 생성
            output = call_openai_chat(system_prompt, item.input["news_article"])
            
            # Trace IO를 업데이트해야 Langfuse의 Eval 기능에서 인풋/아웃풋을 인식합니다.
            root_span.update_trace(input=item.input, output=output)
            
            # 1. 구조적 완성도 평가 (기존)
            structure_score = presence_of_sections_eval(output, item.expected_output.get("sections", []))
            root_span.score_trace(name="section_coverage", value=structure_score)
            
            # 2. LLM-as-a-judge 품질 평가 (새로 추가)
            quality_score = llm_judge_summary_quality(item.input["news_article"], output)
            root_span.score_trace(name="summary_quality", value=quality_score)
            
            # 3. 종합 점수 (구조성 30% + 품질 70%)
            overall_score = (structure_score * 0.3) + (quality_score * 0.7)
            root_span.score_trace(name="overall_score", value=overall_score)
            
            print(f"  - Structure: {structure_score:.3f}, Quality: {quality_score:.3f}, Overall: {overall_score:.3f}")
    
    langfuse.flush()
    print(f"Finished dataset run '{run_name}' on dataset '{DATASET_NAME}'.")

# 두 가지 프롬프트 버전에 대한 평가 실행
print("=" * 60)
print("Starting evaluation for both prompt versions...")
print("=" * 60)

# V0.0.1 (간단 요약) 평가
print("\n🔍 Evaluating V0.0.1 (Simple Summary)...")
run_dataset_experiment(run_name="news_summary_v0.0.1", system_prompt=V001_SIMPLE_SUMMARY)

print("\n" + "=" * 60)

# V0.0.2 (구조화된 요약) 평가  
print("\n🔍 Evaluating V0.0.2 (Structured Summary)...")
run_dataset_experiment(run_name="news_summary_v0.0.2", system_prompt=V002_STRUCTURED_MINUTES)

print("\n" + "=" * 60)
print("✅ Evaluation completed! Check Langfuse dashboard for detailed results.")
print("=" * 60)


Starting evaluation for both prompt versions...

🔍 Evaluating V0.0.1 (Simple Summary)...
Starting dataset run 'news_summary_v0.0.1' with 10 items...
Processing item 1/10...
  - Structure: 0.000, Quality: 0.850, Overall: 0.595
Processing item 2/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 3/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 4/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 5/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 6/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 7/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 8/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Processing item 9/10...
  - Structure: 0.000, Quality: 0.850, Overall: 0.595
Processing item 10/10...
  - Structure: 0.000, Quality: 0.900, Overall: 0.630
Finished dataset run 'news_summary_v0.0.1' on dataset 'week03_news_summary_demo'


## 8) (선택) Langfuse Prompt 버전 생성 & 라벨 지정 (REST API 예시)

- 동일 `promptName`으로 새 버전을 생성하면 자동으로 버전이 증분됩니다.  
- `labels`에 `staging`, `production` 등을 부여/변경하여 배포 포인터로 사용합니다.


In [16]:

import base64, json

def lf_auth_headers():
    token = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()
    return {"Authorization": f"Basic {token}", "Content-Type": "application/json"}

def create_prompt_version_via_rest(prompt_name: str, prompt_text: str, model_name: str, labels=None):
    if not (LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY):
        print("Missing Langfuse credentials.")
        return None
    import requests
    url = f"{LANGFUSE_HOST.rstrip('/')}/api/public/v2/prompts"
    payload = {
        "name": prompt_name,
        "prompt": prompt_text,
        "config": {"model_name": model_name, "temperature": 0.2},
        "labels": labels or []
    }
    r = requests.post(url, headers=lf_auth_headers(), json=payload, timeout=30)
    try:
        r.raise_for_status()
    except Exception as e:
        print("Prompt creation failed:", r.status_code, r.text)
        raise
    return r.json()

# 예시: v0.1 → v0.2 (staging → production) 업로드 (주석 해제 후 사용)
create_prompt_version_via_rest("news-summary", V001_SIMPLE_SUMMARY, "gpt-4o-mini", labels=["staging"])
create_prompt_version_via_rest("news-summary", V002_STRUCTURED_MINUTES, "gpt-4o-mini", labels=["production"])


{'id': '4b6ae362-fe9c-4734-849e-f76d66765ff8',
 'createdAt': '2025-09-21T05:08:59.173Z',
 'updatedAt': '2025-09-21T05:08:59.173Z',
 'projectId': 'cmfl1scks01diad07dc3azk31',
 'createdBy': 'API',
 'prompt': "You are an expert news analyst.\nSummarize the following news article into a **structured Markdown report** with the following sections:\n\n# Headline\n- A concise title (≤15 words) that captures the main story\n\n# Summary\n- 1–2 paragraphs describing the main events in neutral tone\n\n# Key Points\n- 3–5 bullet points with the most critical facts (who, what, when, where, why, how)\n\n# Implications\n- 2–3 sentences on potential impacts (political, economic, or social)\n\n# Quotes\n- Up to 2 direct quotes if available in the article\n\nRules:\n- Remain objective, no speculation beyond the article content.\n- Do not exceed 300 words in total.\n- Preserve named entities (people, organizations, places) accurately.\n- Write in 'Korean'\n",
 'name': 'news-summary',
 'version': 2,
 'type


## 9) Prompty 파일 생성 (GitHub PR용)

- `prompts/meeting_minutes_v0.1.prompty`  
- `prompts/meeting_minutes_v0.2.prompty`  
를 생성합니다. (PR 시 `v0.1 → v0.2` 변경 경험 포함)


In [ ]:

from pathlib import Path
print("Prompty files:")
for p in Path("week03/prompts").glob("*.prompty"):
    print("-", p.name)



## 10) 마무리 & 다음 단계
1. **Tracing 확인**: Langfuse 콘솔에서 오늘 생성된 트레이스를 확인합니다.  
2. **Dataset Run**: `run_dataset_experiment(...)` 실행 후 지표(예: `section_coverage`) 확인.  
3. **Prompt 배포 라벨**: REST 또는 UI로 `staging` → `production` 라벨 전환.  
4. **GitHub PR**: `prompts/meeting_minutes_v0.1.prompty` → `v0.2` 변경 포함 PR 생성.  
5. **초대**: 프로젝트에 `smilechacha@ajou.ac.kr` 뷰어 이상 권한 초대.
